# Initial Data Wrangling
1. #### This notebook explores the raw data given by the the census and Kaggle api's as well as the data downloaded from the Colorado Department of Education. 
2. #### It will guide feature extraction from the datasets to a more human readable format.
3. #### Finally, it will reduce the number of datasets to a more manageable amount to aid in exploratory data analysis. 

In [36]:
import numpy as np
import pandas as pd

# To modules necessary to load the src.data.get_raw_data
import importlib
import sys

# setting path
sys.path.append('..')
# importing
from src.data import get_raw_data
importlib.reload(get_raw_data);

In [33]:
# Load the raw data 
raw_filepath = '../data/raw/' # filepath to all raw data
get_raw_data.get_census(raw_filepath + 'census/') # loads SAIPE data sets from census.gov
get_raw_data.get_kaggle(raw_filepath + 'kaggle/') # obtains the Kaggle competition data